In [1]:
# Setup the Jupyter ve=rsion of Dash
from jupyter_dash import JupyterDash

import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from animal_shelter import AnimalShelter

# Connect to MongoDB
username = "aacuser"
password = "RescuePets2025!"
shelter = AnimalShelter(username, password)


# Retrieve all the record from the database
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


# Setup the app
app = JupyterDash('ProjectTwoDashboard')

# App layout
app.layout = html.Div([
    html.Center(html.Img(src='assets/Grazioso Salvare Logo.png', height='100')), # Grazioso logo
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.P("Jonathan Skop, SNHU CS-340")), #Unique Identifier
    html.Hr(),
    
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Wilderness Rescue', 'value': 'wilderness'},
            {'label': 'Disaster/Tracking', 'value': 'disaster'}
        ],
        value='all',
        labelStyle={'display': 'inline-block', 'margin': '10px'}
    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[]
    ),
             
    html.Br(),
    dcc.Graph(id='bar-chart'), # Second required chart
    html.Br(),         
    html.Div(id='map-id', className='col s12 m6') #Map container
])
             
# Style selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Filtering logic 
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_value):
    if filter_value == 'water':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}     
        }
    
    elif filter_value == 'wilderness':
        query = {
            'breed': {'$in': ['German shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Huskey', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte':156}
        }
    
    elif filter_value == 'disaster':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    
    else:
        query = {}
             
    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')
             
# Map callback
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index): 
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return [html.P("No valid location data to display.")]

    row = index[0] if index else 0 
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[
                dff.loc[row, 'location_lat'],
                dff.loc[row, 'location_long']
            ], children=[
                dl.Tooltip(dff.loc[row, 'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[row, 'name'])
                ])
            ])
        ])
    ]

# Bar chart callback
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_bar_chart(data):
    if not data:
       return {}
    dff = pd.DataFrame(data)
    return px.histogram(dff, x='breed', title='Breed Distribution', color='breed')

# Run App
app.run_server(debug=True)


Dash app running on http://127.0.0.1:17416/
